In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")


: 

In [ ]:
def months_ahead(from_date, to_date):
    """
    Utility: calculate how many months from 'from_date' to 'to_date'.
    Assumes both are month-start if your data is monthly.
    """
    return (to_date.year - from_date.year)*12 + (to_date.month - from_date.month)



In [ ]:
df = pd.read_csv("air-traffic-landings-statistics.csv", encoding='latin1')
df.head()

# Remove outliers from 'Landing Count' and 'Total Landed Weight'
df = remove_outliers_iqr(df, 'Landing Count')
df = remove_outliers_iqr(df, 'Total Landed Weight')


In [ ]:
df = df.apply(lambda x: x.str.lower() if x.dtype == object else x)

df.head()

In [ ]:
df.drop_duplicates(inplace = True)
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.isnull().sum() / len(df)

In [ ]:
df.isna().sum()//len(df)*100

In [ ]:
mean_cleaned = df['Landing Count'].dropna().mean()
print("Mean after dropping missing values:", mean_cleaned)
# Calculate the median of Total Landed Weight
median_total_weight = df['Total Landed Weight'].median()
print("Median Total Landed Weight:", median_total_weight)
# Drop missing values and calculate median
median_cleaned = df['Landing Count'].dropna().median()
print("Median after handling missing values:", median_cleaned)

In [ ]:
# Convert Activity Period (e.g. 201809 -> "2018-09-01")
df['Activity Period'] = df['Activity Period'].astype(str)
df['year'] = df['Activity Period'].str[:4].astype(int)
df['month'] = df['Activity Period'].str[4:].astype(int)
df['date'] = pd.to_datetime(df[['year','month']].assign(day=1))  # always day=1
df.sort_values('date', inplace=True)


In [ ]:
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Summary of landings by month
monthly_landings = df.groupby('Activity Period')['Landing Count'].sum()
monthly_landings.plot(kind='line', title='Monthly Landings', xlabel='Month', ylabel='Landings')
plt.show()

In [ ]:
# Top operating airlines by landing count
top_airlines = df.groupby('Operating Airline')['Landing Count'].sum().sort_values(ascending=False).head(10)
top_airlines.plot(kind='bar', title='Top Airlines by Landings', xlabel='Airline', ylabel='Landings')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# Aircraft body type distribution
body_type_distribution = df['Aircraft Body Type'].value_counts()
body_type_distribution.plot(kind='pie', autopct='%1.1f%%', title='Aircraft Body Type Distribution')
plt.show()

In [ ]:
# Weight trend over time
df['Total Landed Weight'] = pd.to_numeric(df['Total Landed Weight'], errors='coerce')
weight_trend = df.groupby('Activity Period')['Total Landed Weight'].sum()
weight_trend.plot(kind='line', title='Total Landed Weight Over Time', xlabel='Month', ylabel='Weight')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Boxplot for landing counts by aircraft body type
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Aircraft Body Type', y='Landing Count')
plt.title('Distribution of Landing Counts by Aircraft Body Type')
plt.xlabel('Aircraft Body Type')
plt.ylabel('Landing Count')
plt.show()

In [ ]:
# Scatterplot for total landed weight vs. landing coun
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Landing Count', y='Total Landed Weight', hue='Aircraft Body Type', alpha=0.7)
plt.title('Scatterplot of Total Landed Weight vs. Landing Count')
plt.xlabel('Landing Count')
plt.ylabel('Total Landed Weight')
plt.legend(title='Aircraft Body Type')
plt.show()

In [ ]:
# Histogram with KDE
plt.figure(figsize=(10, 6))
sns.histplot(df['Landing Count'], bins=20, kde=True, color='green')
plt.title('Histogram of Landing Counts with KDE')
plt.xlabel('Landing Count')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Top airlines by landing count
top_airlines = df.groupby('Operating Airline')['Landing Count'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
top_airlines.plot(kind='bar', color='gold', edgecolor='black')
plt.title('Top Airlines by Landings')
plt.xlabel('Operating Airline')
plt.ylabel('Total Landings')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
numerical_features = df[['Landing Count', 'Total Landed Weight']]
correlation_matrix = numerical_features.corr()

# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap of Correlation Between Numerical Features')
plt.show()

In [ ]:
df_monthly = df.groupby('date', as_index=False).agg({
    'Landing Count': 'sum',
    'Total Landed Weight': 'sum'
})
# Set index to date, ensure monthly frequency
df_monthly.set_index('date', inplace=True)
df_monthly = df_monthly.asfreq('MS')  # 'MS' = Month Start


In [ ]:
# import pandas as pd

# def remove_outliers_iqr(df, column):
#     """
#     Remove outliers from a DataFrame column using the IQR method.
#     """
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1

#     # Define outlier bounds
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR

#     # Filter out outliers
#     df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
#     return df_filtered
# print(f"Original data points: {len(df_monthly)}")
# # Apply outlier removal on both Landing Count and Total Landed Weight
# df_monthly = remove_outliers_iqr(df_monthly, 'Landing Count')
# df_monthly = remove_outliers_iqr(df_monthly, 'Total Landed Weight')


# print(f"Data points after outlier removal: {len(df_monthly)}")

In [ ]:
# You can choose a train end date, or just use the full dataset.
train_data_lc = df_monthly['Landing Count']
train_data_tlw = df_monthly['Total Landed Weight']

In [ ]:
model_lc = SARIMAX(
    train_data_lc,
    order=(1,1,1),              # (p,d,q) - placeholder, tune for your data
    seasonal_order=(1,1,1,12),  # (P,D,Q,m) with m=12 for monthly seasonality
    enforce_stationarity=False,
    enforce_invertibility=False
)
results_lc = model_lc.fit(disp=False)


In [ ]:

# ---- SARIMAX for Total Landed Weight ----
model_tlw = SARIMAX(
    train_data_tlw,
    order=(1,1,1),
    seasonal_order=(1,1,1,12),
    enforce_stationarity=False,
    enforce_invertibility=False
)
results_tlw = model_tlw.fit(disp=False)


In [ ]:

future_date_str = '2025-03-01'
future_date = pd.to_datetime(future_date_str)

last_date_in_data = df_monthly.index.max()
steps = months_ahead(last_date_in_data, future_date)
if steps <= 0:
    raise ValueError("Future date is not after the dataset's last date!")

# Forecast Landing Count
forecast_lc = results_lc.get_forecast(steps=steps)
pred_lc_series = forecast_lc.predicted_mean
predicted_lc = pred_lc_series.loc[future_date]

# Forecast Total Landed Weight
forecast_tlw = results_tlw.get_forecast(steps=steps)
pred_tlw_series = forecast_tlw.predicted_mean
predicted_tlw = pred_tlw_series.loc[future_date]

print(f"\n=== Forecast for {future_date_str} ===")
print(f"Landing Count: {predicted_lc:.0f}")
print(f"Total Landed Weight: {predicted_tlw:.0f}")


In [ ]:
df_latest_month = df[df['date'] == last_date_in_data]
if df_latest_month.empty:
    raise ValueError(f"No data found for last_date_in_data={last_date_in_data}. Adjust as needed.")

# Summation by airline
airline_sums = df_latest_month.groupby('Operating Airline')['Landing Count'].sum()
airline_proportions = airline_sums / airline_sums.sum()

# Distribute predicted_lc across airlines
airline_lc_forecast = airline_proportions * predicted_lc

# Similarly, we can distribute predicted_tlw across airlines
airline_tlw_forecast = airline_proportions * predicted_tlw

# -----------------------------
# Step B: For each airline, distribute by Aircraft Body Type
# -----------------------------
# We look at that airline's distribution of body types in the last actual month
df_latest_month_airline_body = (
    df_latest_month
    .groupby(['Operating Airline','Aircraft Body Type'])['Landing Count']
    .sum()
)

predicted_rows = []

for airline, airline_share_lc in airline_lc_forecast.items():
    # The portion of total landed weight for this airline
    airline_share_tlw = airline_tlw_forecast.get(airline, 0.0)

    # Filter to just this airline in the last month
    # to see distribution of body types
    sub = df_latest_month_airline_body.loc[airline] if airline in df_latest_month_airline_body.index.levels[0] else None

    if sub is not None and isinstance(sub, pd.Series) and not sub.empty:
        # sub is a Series with index = body types, value = landing count
        body_type_props = sub / sub.sum()  # proportion of each body type within this airline
        for btype, prop_val in body_type_props.items():
            final_lc = airline_share_lc * prop_val
            final_tlw = airline_share_tlw * prop_val

            predicted_rows.append({
                'Activity Period': future_date.strftime('%Y%m'),
                'date': future_date,
                'Operating Airline': airline,
                'Landing Count': final_lc,
                'Total Landed Weight': final_tlw
            })
    else:
        # If there's no breakdown for this airline, we can store a single row with no body type
        predicted_rows.append({
            'Activity Period': future_date.strftime('%Y%m'),
            'date': future_date,
            'Operating Airline': airline,
            'Landing Count': airline_share_lc, 
            'Total Landed Weight': airline_share_tlw
        })

# Convert to DataFrame
predicted_df = pd.DataFrame(predicted_rows)

# =========================================================
# 6) SHOW FINAL PREDICTED DATAFRAME
#    Each row = (Airline, Body Type) with forecasted LC, TLW
# =========================================================

print("\n=== Final Predicted Rows for 2025-02 (Airline x Body Type) ===")
print(predicted_df.head(20))  # Show first 20 rows


In [ ]:
in_sample_pred_lc = results_lc.get_prediction(start=train_data_lc.index[0],
                                               end=train_data_lc.index[-1]).predicted_mean
actual_lc = train_data_lc

rmse_lc = np.sqrt(mean_squared_error(actual_lc, in_sample_pred_lc))
mae_lc = mean_absolute_error(actual_lc, in_sample_pred_lc)
print(f"\n--- In-Sample Error Metrics for Landing Count ---")
print(f"RMSE: {rmse_lc:.2f}")
print(f"MAE: {mae_lc:.2f}")

# Create a continuous time series that combines in-sample predictions and forecast.
predicted_full_lc = pd.concat([in_sample_pred_lc, pred_lc_series])

plt.figure(figsize=(12, 6))
plt.plot(train_data_lc.index, train_data_lc, label='Actual Landing Count', color='blue')
plt.plot(in_sample_pred_lc.index, in_sample_pred_lc, label='Fitted (In-Sample)', color='red', alpha=0.7)
plt.plot(pred_lc_series.index, pred_lc_series, label='Forecast', color='green', linestyle='--', linewidth=2)
plt.title(f"Landing Count Forecast\nIn-Sample RMSE: {rmse_lc:.2f}, MAE: {mae_lc:.2f}")
plt.xlabel("Date")
plt.ylabel("Landing Count")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

def remove_outliers_iqr(df, column):
    """
    Remove outliers from a DataFrame column using the IQR method.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out outliers
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

# Apply outlier removal on both Landing Count and Total Landed Weight
df_no_outliers = remove_outliers_iqr(df_monthly, 'Landing Count')
df_no_outliers = remove_outliers_iqr(df_no_outliers, 'Total Landed Weight')

print(f"Original data points: {len(df_monthly)}")
print(f"Data points after outlier removal: {len(df_no_outliers)}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to plot boxplots and time series
def visualize_outliers(original_df, cleaned_df, column_name):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Outlier Analysis for {column_name}', fontsize=16)

    # Boxplot before outlier removal
    sns.boxplot(data=original_df, y=column_name, ax=axes[0, 0])
    axes[0, 0].set_title('Before Outlier Removal - Boxplot')

    # Boxplot after outlier removal
    sns.boxplot(data=cleaned_df, y=column_name, ax=axes[0, 1])
    axes[0, 1].set_title('After Outlier Removal - Boxplot')

    # Time series before outlier removal
    axes[1, 0].plot(original_df.index, original_df[column_name], label='Original', color='blue')
    axes[1, 0].set_title('Before Outlier Removal - Time Series')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel(column_name)

    # Time series after outlier removal
    axes[1, 1].plot(cleaned_df.index, cleaned_df[column_name], label='Cleaned', color='green')
    axes[1, 1].set_title('After Outlier Removal - Time Series')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel(column_name)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Apply visualization for Landing Count
visualize_outliers(df_monthly, df_no_outliers, 'Landing Count')

# Apply visualization for Total Landed Weight
visualize_outliers(df_monthly, df_no_outliers, 'Total Landed Weight')

